### TTN generation sandbox

New method:
Instead of using pyGeo, we now just use regular pyTorch.

To output the graph, we'll interpret an output vector as probabilities in a graph.
Post-processing probably required for:
- ensuring graph is connected
- ensuring graph is tree-like

In [40]:
# %automagic

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from matplotlib import pyplot as plt

import pickle

# import importlib
# ttn_package = importlib.import_module("../ttn_demo/ttn.py")

# from ..ttn_demo import ttn

import ttn

In [105]:


input_dim = 2
output_dim = 2

# Define the model class
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        # First hidden layer
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        # Second hidden layer
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        # Output layer
        self.fc3 = nn.Linear(hidden_dim, output_dim * output_dim)

    def forward(self, x):
        # Pass through hidden layers with activation
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        # Output layer without activation (linear regression)
        return self.fc3(x).view(x.shape[0], output_dim, output_dim)

# Sample training data (modify this based on your actual data)
input_data = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
output_data = torch.tensor([ [[3.0, 4.0],[3.0, 4.0]],
                             [[7.0, 8.0], [7.0, 8.0]],
                             [[11.0, 12.0],[11.0, 12.0]]  ]  )

# Create a dataset and dataloader
dataset = TensorDataset(input_data, output_data)
dataloader = DataLoader(dataset, batch_size=2)  # Adjust batch_size as needed

# Define model, loss function, and optimizer
model = MLP(input_data.shape[1], 16, output_data.shape[1])  # Adjust hidden layer size
loss_fn = nn.MSELoss()  # Mean Squared Error loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

# Train the model
for epoch in range(100):  # Adjust number of epochs for better training
    for data, target in dataloader:
        optimizer.zero_grad()
        prediction = model(data)
        loss = loss_fn(prediction, target)
        loss.backward()
        optimizer.step()

# Example usage: predict output for a new input
new_input = torch.tensor([[7.0, 8.0]])
predicted_output = model(new_input)
print(f"Predicted output for new input:\n {predicted_output}")

NameError: name 'output_dim' is not defined

In [54]:
'''
Matt`s TTN data is in "/Users/nicolassawaya/proj/pando/pando-TN-repo/ttn_learning/data/learning_pairs"
'''
# path_ttn_data = "/Users/nicolassawaya/proj/pando/pando-TN-repo/ttn_learning/data/learning_pairs/"
path_ttn_data = "../ttn_learning/data/learning_pairs/"
fname_ttn_L16 = path_ttn_data+"pairs_L16_D2_m3.pkl"
fname_ttn_L24 = path_ttn_data+"pairs_L24_D2_m3.pkl"

with open(fname_ttn_L16, "rb") as f:
    data_L16 = pickle.load(f)
with open(fname_ttn_L24, "rb") as f:
    data_L24 = pickle.load(f)
    
print(data_L16)


[(array([-0.30894907,  1.96295376,  0.86974369, -0.27542627,  2.19114059,
        1.85649599,  1.64596818,  1.65652284,  2.28712287,  1.94617553,
        1.70348118,  2.50535126,  1.62462432,  0.6268707 ,  1.37940695,
        0.35255629]), TTN(tensors=31, indices=46)), (array([2.45673348, 0.32151607, 2.20356097, 1.30259196, 1.41356572,
       2.94043924, 1.39695988, 1.16426218, 0.83298394, 1.56836296,
       1.56442525, 2.92932993, 0.80692259, 0.14826019, 1.57305043,
       0.79891753]), TTN(tensors=31, indices=46)), (array([ 0.05627459, -0.5247911 ,  2.23555308,  0.63961334,  2.31403569,
        1.21732606,  0.7591391 ,  0.69600629, -1.18423597,  0.54727062,
        3.74493231,  1.4457932 ,  1.63832917,  0.47105671,  1.74910897,
        1.57566462]), TTN(tensors=31, indices=46)), (array([-0.20873274,  2.11234539,  0.44366383,  2.69534072,  1.02803822,
        0.87226619, -1.19261869,  0.19342916,  1.49592225,  2.104682  ,
       -0.97143788, -0.32822398,  0.0639892 ,  1.10261315, -0.3

In [64]:
adjmat = data_L24[8][1].get_adjacency_matrix()

# plt.spy( adjmat )
print(adjmat[:,2])
print( adjmat.max() )

print(type(data_L24[0][0]))
print(type(data_L24[0][1]))

list( data_L24[0][0] )
list( data_L24[0][1].get_adjacency_matrix() )


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
3
<class 'numpy.ndarray'>
<class 'ttn.TTN'>


[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=int16),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=int16),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=int16),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=int16),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=int16),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 

In [111]:
def matthew_to_torch(learning_pairs):
    """Takes Matt's 'learning pairs' data into pytorch format"""
    
    # input_data = torch.tensor( [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    # output_data = torch.tensor([ [[3.0, 4.0], [3.0, 4.0]],
    #                              [[7.0, 8.0], [7.0, 8.0]],
    #                              [[11.0, 12.0], [11.0, 12.0]]  ]  )
    
    list_couplings = []
    list_adjmats = []

    # inp_tensor = torch.tensor()
    # out_tensor = torch.tensor()

    ctr=0
    for pair in learning_pairs:
        list_couplings.append( torch.tensor(pair[0]) )
        list_adjmats.append( torch.tensor(pair[1].get_adjacency_matrix()) )
        # ctr += 1
        # if ctr>2:
        #     break
    
    return torch.stack(list_couplings), torch.stack(list_adjmats)
    

In [101]:
inpdat,outdat = matthew_to_torch(data_L16)
print(inpdat.shape)
print(outdat.shape)
print(inpdat.dtype)
print(outdat.dtype)
inpdat = inpdat.to(torch.float32)
outdat = outdat.to(torch.float32)
print(inpdat.dtype)
print(outdat.dtype)
print(inpdat)
print(outdat)

torch.Size([56, 16])
torch.Size([56, 31, 31])
torch.float64
torch.int16
torch.float32
torch.float32
tensor([[-3.0895e-01,  1.9630e+00,  8.6974e-01, -2.7543e-01,  2.1911e+00,
          1.8565e+00,  1.6460e+00,  1.6565e+00,  2.2871e+00,  1.9462e+00,
          1.7035e+00,  2.5054e+00,  1.6246e+00,  6.2687e-01,  1.3794e+00,
          3.5256e-01],
        [ 2.4567e+00,  3.2152e-01,  2.2036e+00,  1.3026e+00,  1.4136e+00,
          2.9404e+00,  1.3970e+00,  1.1643e+00,  8.3298e-01,  1.5684e+00,
          1.5644e+00,  2.9293e+00,  8.0692e-01,  1.4826e-01,  1.5731e+00,
          7.9892e-01],
        [ 5.6275e-02, -5.2479e-01,  2.2356e+00,  6.3961e-01,  2.3140e+00,
          1.2173e+00,  7.5914e-01,  6.9601e-01, -1.1842e+00,  5.4727e-01,
          3.7449e+00,  1.4458e+00,  1.6383e+00,  4.7106e-01,  1.7491e+00,
          1.5757e+00],
        [-2.0873e-01,  2.1123e+00,  4.4366e-01,  2.6953e+00,  1.0280e+00,
          8.7227e-01, -1.1926e+00,  1.9343e-01,  1.4959e+00,  2.1047e+00,
         -9.7144e

In [125]:

input_dim  = inpdat.shape[1]
output_dim = outdat.shape[1]
print(input_dim)
print(output_dim)

# Define the model class
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        # First hidden layer
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        # Second hidden layer
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        # Output layer
        self.fc3 = nn.Linear(hidden_dim, output_dim * output_dim)

    def forward(self, x):
        # Pass through hidden layers with activation
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        # Output layer without activation (linear regression)
        return self.fc3(x).view(x.shape[0], output_dim, output_dim)

# Create a dataset and dataloader
dataset = TensorDataset(inpdat, outdat)
dataloader = DataLoader(dataset, batch_size=4)  # Adjust batch_size as needed

# Define model, loss function, and optimizer
model = MLP(inpdat.shape[1], 16, outdat.shape[1])  # Adjust hidden layer size
loss_fn = nn.MSELoss()  # Mean Squared Error loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

# Train the model
for epoch in range(100):  # Adjust number of epochs for better training
    for data, target in dataloader:
        optimizer.zero_grad()
        prediction = model(data)
        loss = loss_fn(prediction, target)
        loss.backward()
        optimizer.step()

# Example usage: predict output for a new input
new_input = torch.tensor([list(range(16))]).to(torch.float32)
predicted_output = model(new_input)
print(f"Predicted output for new input:\n {predicted_output}")


16
31
Predicted output for new input:
 tensor([[[-7.7747e-03,  2.1380e-01, -1.6585e-01, -1.0024e-01,  1.6139e-01,
          -6.0152e-02, -1.1579e-01, -7.3285e-02, -2.7445e-01,  2.8841e-01,
          -3.3634e-01, -1.5521e-01, -1.2954e-01,  1.1629e-01, -4.5117e-01,
           1.9926e-01,  3.8311e-02,  2.7376e-01, -3.3543e-01,  4.8679e-01,
           1.8267e-01,  4.2103e-02, -2.7377e-01,  5.4501e-01,  1.2434e-01,
           2.9306e-01,  3.9964e-01, -4.1127e-02, -1.9464e-02,  3.5240e-01,
          -2.2735e-01],
         [-5.3218e-01,  5.2189e-02, -3.8545e-01,  4.3338e-01,  1.0758e-01,
          -2.4636e-01, -2.5366e-01,  3.8107e-03,  1.5976e-01,  4.0606e-01,
          -2.5935e-01,  3.3557e-02,  2.9346e-01,  5.3315e-02,  2.2922e-01,
           2.6410e-01, -1.5339e-01,  3.4642e-01,  5.7444e-05,  3.1129e-01,
           4.6756e-01, -2.5541e-01, -5.8440e-02, -3.5606e-01, -2.6080e-01,
          -1.6698e-01,  1.7248e-01,  2.0310e-01,  3.3687e-01, -2.7141e-01,
           2.2077e-01],
         [ 5.